In [6]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import inv
import matplotlib.pyplot as plt
import os

In [11]:
wavelengths = np.arange(380, 781)
len(wavelengths)

401

In [12]:
class SPDReNMF:
    def __init__(self,spd_data):
        self.data = np.clip(spd_data, 0, None)
        self.wavelengths = np.arange(380, 780)
        self.mu = np.array([415, 445, 480, 515, 555, 590, 630, 680])
        self.sigma = np.array([11.0403, 12.7388, 15.2866, 16.5605, 16.5605, 16.9851, 21.2314, 22.0807])
        

    def gaussian_filter(self):
        filters = np.zeros((len(self.mu), len(self.wavelengths)))
        for i in range(len(self.mu)):
            filters[i] = np.exp(-0.5 * ((self.wavelengths - self.mu[i]) / self.sigma[i])**2)
        self.M = filters
    
    def compute_R(self):
        self.R = np.dot(self.M, self.data)
    
    def compute_H(self):
        nmf = NMF(n_components=8) 
        scores = nmf.fit_transform(self.data) 
        self.H = scores.T
    
    def compute_V(self):
        self.MH_inv = np.linalg.inv(np.dot(self.M, self.H.T))
        self.V = np.dot(self.MH_inv,self.R)

    def Reconstructed_spectrum(self):
        self.reconstructed_spectrum = np.dot(self.H.T, self.V)
        
    
    def Evaluate(self, Light, save_path=None):
        s = self.data[:, Light]
        s_re = self.reconstructed_spectrum[:, Light]
        MAE = np.mean(np.absolute(s - s_re))
        RMSE = np.sqrt(np.mean(np.square(s - s_re)))
        RRMSE = RMSE / np.mean(s_re)

        # GFC
        num = abs(np.sum(s * s_re))
        denom_s = np.sqrt(np.sum(s ** 2) + 1e-9)
        denom_s_re = np.sqrt(np.sum(s_re ** 2) + 1e-9)
        GFC = num / (denom_s * denom_s_re)

        print("MAE: ", MAE)
        print("RMSE: ", RMSE)
        print("RRMES: ", RRMSE)
        print("GFC: ", GFC)

        if save_path:
            with open(save_path, 'a') as f:
                f.write(f"Light: {Light + 1}\n")
                f.write(f"MAE: {MAE}\n")
                f.write(f"RMSE: {RMSE}\n")
                f.write(f"RRMES: {RRMSE}\n")
                f.write(f"GFC: {GFC}\n\n")
            print(f"Results saved to {save_path}")
        

    def Plot(self, Light, save_path=None):
        # 创建一个包含三个子图的图像
        fig, axs = plt.subplots(3, 1)
        s = self.data[:, Light]
        r = self.reconstructed_spectrum[:, Light]

        # 绘制真实光谱图
        axs[0].plot(self.wavelengths, s, label='True Spectrum')
        axs[0].set_xlabel('Wavelength (nm)')
        axs[0].set_ylabel('Intensity')
        axs[0].set_title('True Spectrum vs Reconstructed Spectrum')
        axs[0].legend()

        # 绘制重建光谱图
        axs[1].plot(self.wavelengths, r, label='Reconstructed Spectrum')
        axs[1].set_xlabel('Wavelength (nm)')
        axs[1].set_ylabel('Intensity')
        axs[1].legend()

        # 绘制重建光谱和真实光谱叠加的图
        axs[2].plot(self.wavelengths, s, label='True Spectrum')
        axs[2].plot(self.wavelengths, r, label='Reconstructed Spectrum')
        axs[2].set_xlabel('Wavelength (nm)')
        axs[2].set_ylabel('Intensity')
        axs[2].legend()

        if save_path:
            file_name = f"Light_s{Light + 1}.png"
            plt.savefig(os.path.join(save_path, file_name))

        plt.tight_layout()
        plt.show()

In [27]:
spd = np.load("denoised_data.npy", allow_pickle= True)
spd.shape

(5661, 400)

In [8]:
scaler = MinMaxScaler()
scalered_data = scaler.fit_transform(spd)
scalered_data.shape

(400, 5661)

In [13]:
r = SPDReNMF(spd)

In [14]:
r.gaussian_filter()
r.compute_R()
r.compute_H()
r.compute_V()
r.Reconstructed_spectrum()
print("M: ", r.M.shape)
print("R: ", r.R.shape)
print("H: ", r.H.shape)
print("V: ", r.V.shape)
print("s: ", r.reconstructed_spectrum.shape)

M:  (8, 400)
R:  (8, 5661)
H:  (8, 400)
V:  (8, 5661)
s:  (400, 5661)


/opt/anaconda3/envs/ForEXE/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [18]:
test_result = r.reconstructed_spectrum

t = scaler.inverse_transform(test_result)

t.shape

(400, 5661)

In [22]:
wavelengths = np.arange(380, 780, 1)

def Plot(Light, save_path=None):
    # 创建一个包含三个子图的图像
    fig, axs = plt.subplots(3, 1)
    s = spd[:, Light]
    r = test_result[:, Light]

    # 绘制真实光谱图
    axs[0].plot(wavelengths, s, label='True Spectrum')
    axs[0].set_xlabel('Wavelength (nm)')
    axs[0].set_ylabel('Intensity')
    axs[0].set_title('True Spectrum vs Reconstructed Spectrum')
    axs[0].legend()

        # 绘制重建光谱图
    axs[1].plot(wavelengths, r, label='Reconstructed Spectrum')
    axs[1].set_xlabel('Wavelength (nm)')
    axs[1].set_ylabel('Intensity')
    axs[1].legend()

    # 绘制重建光谱和真实光谱叠加的图
    axs[2].plot(wavelengths, s, label='True Spectrum')
    axs[2].plot(wavelengths, r, label='Reconstructed Spectrum')
    axs[2].set_xlabel('Wavelength (nm)')
    axs[2].set_ylabel('Intensity')
    axs[2].legend()

    if save_path:
        file_name = f"Light_s{Light + 1}.png"
        plt.savefig(os.path.join(save_path, file_name))

    plt.tight_layout()
    plt.show()